<a href="https://colab.research.google.com/github/ashref83/First/blob/master/Copy_of_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import torch

In [21]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [22]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [23]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
targets

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [24]:
w=torch.randn(2,3,requires_grad=True)
b=torch.randn(2,requires_grad=True)

In [25]:
print(w,'\n',b)

tensor([[-0.5859,  0.8639,  0.2347],
        [ 0.0768, -0.4160, -0.6360]], requires_grad=True) 
 tensor([0.9133, 0.7219], requires_grad=True)


In [26]:
def model(x):
    return x @ w.t() + b

In [27]:
preds=model(inputs)

In [28]:
preds

tensor([[ 26.1172, -48.8857],
        [ 38.6419, -69.5935],
        [ 79.3144, -85.2196],
        [-13.0133, -32.8580],
        [ 59.8498, -78.4277]], grad_fn=<AddBackward0>)

In [29]:
def mse (t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [30]:
loss = mse(preds,targets)

In [31]:
loss

tensor(14206.3594, grad_fn=<DivBackward0>)

In [32]:
loss.backward()

In [33]:
w.grad, b.grad

(tensor([[ -3207.4800,  -3339.1045,  -2122.7297],
         [-12787.1602, -14835.1904,  -9018.2988]]),
 tensor([ -38.0180, -154.9969]))

In [34]:
w ,b

(tensor([[-0.5859,  0.8639,  0.2347],
         [ 0.0768, -0.4160, -0.6360]], requires_grad=True),
 tensor([0.9133, 0.7219], requires_grad=True))

In [35]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [36]:
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [37]:
print (loss)

tensor(177.5983, grad_fn=<DivBackward0>)


In [38]:
177 **.5

13.30413469565007

In [39]:
print (preds, targets)

tensor([[ 57.1340,  76.2142],
        [ 79.4398,  97.9340],
        [125.0393, 129.6902],
        [ 19.7339,  69.9512],
        [ 98.0734,  95.3012]], grad_fn=<AddBackward0>) tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [40]:
!pip install jovian --upgrade -q

     |████████████████████████████████| 102kB 2.5MB/s 


In [44]:
import jovian

In [45]:
jovian.commit()

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..


In [46]:
import torch.nn as nn

In [88]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')


In [89]:

targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

In [90]:
from torch.utils.data import TensorDataset

In [91]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [92]:
train_d = TensorDataset(inputs,targets)

In [93]:
from torch.utils.data import DataLoader

In [94]:
batc_size = 5
train_dl = DataLoader(train_d, batch_size=batc_size, shuffle=True)

In [95]:
model = nn.Linear(3,2)

In [96]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2724,  0.4685,  0.3038],
         [-0.3473,  0.3404, -0.5221]], requires_grad=True),
 Parameter containing:
 tensor([0.4371, 0.4966], requires_grad=True)]

In [97]:
preds=model(inputs)

In [98]:
import torch.nn.functional as F

In [99]:
loss_fn = F.mse_loss

In [100]:
loss = loss_fn(preds,targets)

In [101]:
loss

tensor(8646.4023, grad_fn=<MseLossBackward>)

In [108]:
opt=torch.optim.SGD(model.parameters(), lr=1e-5)

In [111]:
def fit (num_epochs, model, loss_fn, opt, train_dl):

  for epoch in range(num_epochs):

    for xa,ya in train_dl:

      preds= model(xa)

      loss = loss_fn(preds,ya)

      loss.backward()

      opt.step()

      opt.zero_grad()

      # Print the progress
    if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))



In [112]:
fit(100,model,loss_fn,opt, train_dl)

Epoch [10/100], Loss: 117.5224
Epoch [20/100], Loss: 51.8352
Epoch [30/100], Loss: 54.0714
Epoch [40/100], Loss: 50.0525
Epoch [50/100], Loss: 39.3213
Epoch [60/100], Loss: 86.7705
Epoch [70/100], Loss: 47.7915
Epoch [80/100], Loss: 63.5707
Epoch [90/100], Loss: 49.9555
Epoch [100/100], Loss: 48.4147


In [113]:
import jovian

In [114]:
jovian.commit()

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..
